# OpenAI APIs - Vision

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/guides/vision).
This tutorial covers the vision APIs for vision language models.

SGLang supports various vision language models such as Llama 3.2, LLaVA-OneVision, Qwen2.5-VL, Gemma3 and [more](https://docs.sglang.ai/references/supported_models): 
- [meta-llama/Llama-3.2-11B-Vision-Instruct](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct)  
- [lmms-lab/llava-onevision-qwen2-72b-ov-chat](https://huggingface.co/lmms-lab/llava-onevision-qwen2-72b-ov-chat)  
- [Qwen/Qwen2.5-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct)
- [google/gemma-3-4b-it](https://huggingface.co/google/gemma-3-4b-it)
- [openbmb/MiniCPM-V](https://huggingface.co/openbmb/MiniCPM-V)
- [deepseek-ai/deepseek-vl2](https://huggingface.co/deepseek-ai/deepseek-vl2)

As an alternative to the OpenAI API, you can also use the [SGLang offline engine](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py).

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

**Remember to add** `--chat-template llama_3_vision` **to specify the [vision chat template](https://docs.sglang.ai/backend/openai_api_vision.html#Chat-Template), otherwise, the server will only support text (images won’t be passed in), which can lead to degraded performance.**

We need to specify `--chat-template` for vision language models because the chat template provided in Hugging Face tokenizer only supports text.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

vision_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-11B-Vision-Instruct \
    --chat-template=llama_3_vision
"""
)

wait_for_server(f"http://localhost:{port}")

INFO 03-26 17:47:57 __init__.py:190] Automatically detected platform cuda.


[2025-03-26 17:48:00] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-11B-Vision-Instruct', tokenizer_path='meta-llama/Llama-3.2-11B-Vision-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-11B-Vision-Instruct', chat_template='llama_3_vision', completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=31027, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=618964215, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', 

[2025-03-26 17:48:05] The following error message 'operation scheduled before its operands' can be ignored.


[2025-03-26 17:48:11] Use chat template for the OpenAI-compatible API server: llama_3_vision


INFO 03-26 17:48:13 __init__.py:190] Automatically detected platform cuda.
INFO 03-26 17:48:13 __init__.py:190] Automatically detected platform cuda.


...

[2025-03-26 17:48:32 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-03-26 17:48:33] INFO:     Started server process [485340]
[2025-03-26 17:48:33] INFO:     Waiting for application startup.
[2025-03-26 17:48:33] INFO:     Application startup complete.
[2025-03-26 17:48:33] INFO:     Uvicorn running on http://127.0.0.1:31027 (Press CTRL+C to quit)


[2025-03-26 17:48:33] INFO:     127.0.0.1:59148 - "GET /v1/models HTTP/1.1" 200 OK


[2025-03-26 17:48:34] INFO:     127.0.0.1:59160 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-03-26 17:48:34 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-26 17:48:36] INFO:     127.0.0.1:59166 - "POST /generate HTTP/1.1" 200 OK
[2025-03-26 17:48:36] The server is fired up and ready to roll!


## Using cURL

Once the server is up, you can send test requests using curl or requests.

In [2]:
import subprocess

curl_command = f"""
curl -s http://localhost:{port}/v1/chat/completions \\
  -d '{{
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
      {{
        "role": "user",
        "content": [
          {{
            "type": "text",
            "text": "What’s in this image?"
          }},
          {{
            "type": "image_url",
            "image_url": {{
              "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
            }}
          }}
        ]
      }}
    ],
    "max_tokens": 300
  }}'
"""

response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)


response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)

[2025-03-26 17:48:44 TP0] Prefill batch. #new-seq: 1, #new-token: 6463, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-26 17:48:45 TP0] Decode batch. #running-req: 1, #token: 6496, token usage: 0.32, gen throughput (token/s): 3.13, #queue-req: 0, 


[2025-03-26 17:48:46 TP0] Decode batch. #running-req: 1, #token: 6536, token usage: 0.32, gen throughput (token/s): 67.47, #queue-req: 0, 


[2025-03-26 17:48:46 TP0] Decode batch. #running-req: 1, #token: 6576, token usage: 0.32, gen throughput (token/s): 68.23, #queue-req: 0, 
[2025-03-26 17:48:46] INFO:     127.0.0.1:33864 - "POST /v1/chat/completions HTTP/1.1" 200 OK


[2025-03-26 17:48:47 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 6462, token usage: 0.32, #running-req: 0, #queue-req: 0, 


[2025-03-26 17:48:47] INFO:     127.0.0.1:45358 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using Python Requests

In [3]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    "max_tokens": 300,
}

response = requests.post(url, json=data)
print_highlight(response.text)

[2025-03-26 17:48:48 TP0] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 6462, token usage: 0.32, #running-req: 0, #queue-req: 0, 
[2025-03-26 17:48:48 TP0] Decode batch. #running-req: 1, #token: 6466, token usage: 0.32, gen throughput (token/s): 19.10, #queue-req: 0, 


[2025-03-26 17:48:49 TP0] Decode batch. #running-req: 1, #token: 6506, token usage: 0.32, gen throughput (token/s): 68.49, #queue-req: 0, 


[2025-03-26 17:48:49] INFO:     127.0.0.1:45374 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using OpenAI Python Client

In [4]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print_highlight(response.choices[0].message.content)

[2025-03-26 17:48:50 TP0] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 6452, token usage: 0.32, #running-req: 0, #queue-req: 0, 


[2025-03-26 17:48:50 TP0] Decode batch. #running-req: 1, #token: 6474, token usage: 0.32, gen throughput (token/s): 31.73, #queue-req: 0, 


[2025-03-26 17:48:50] INFO:     127.0.0.1:45386 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Multiple-Image Inputs

The server also supports multiple images and interleaved text and images if the model supports it.

In [5]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/sgl-project/sglang/main/assets/logo.png",
                    },
                },
                {
                    "type": "text",
                    "text": "I have two very different images. They are not related at all. "
                    "Please describe the first image in one sentence, and then describe the second image in another sentence.",
                },
            ],
        }
    ],
    temperature=0,
)

print_highlight(response.choices[0].message.content)

[2025-03-26 17:48:51 TP0] Prefill batch. #new-seq: 1, #new-token: 12895, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-03-26 17:48:52 TP0] Decode batch. #running-req: 1, #token: 12917, token usage: 0.63, gen throughput (token/s): 20.56, #queue-req: 0, 


[2025-03-26 17:48:53 TP0] Decode batch. #running-req: 1, #token: 12957, token usage: 0.63, gen throughput (token/s): 63.66, #queue-req: 0, 


[2025-03-26 17:48:53] INFO:     127.0.0.1:45388 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(vision_process)

[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=485712


[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=487360
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and shutting down...
[2025-03-26 17:48:53] SIGTERM received. signum=None frame=None. Draining requests and sh

[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=487381
  File "/public_sglang_ci/runner-a-gpu-0/_work/_tool/Python/3.9.21/x64/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/public_sglang_ci/runner-a-gpu-0/_work/_tool/Python/3.9.21/x64/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/public_sglang_ci/runner-a-gpu-0/_work/sglang/sglang/python/sglang/launch_server.py", line 14, in <module>
    launch_server(server_args)
  File "/public_sglang_ci/runner-a-gpu-0/_work/sglang/sglang/python/sglang/srt/entrypoints/http_server.py", line 708, in launch_server
    uvicorn.run(
  File "/public_sglang_ci/runner-a-gpu-0/_work/_tool/Python/3.9.21/x64/lib/python3.9/site-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/public_sglang_ci/runner-a-gpu-0/_work/_tool/Python/3.9.21/x64/lib/python3.9/site-packages/uvicorn/server.py", line 66, in run
    re

[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=487386
[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=487396
[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=487355
[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=485714
[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=487352
[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=487358
[2025-03-26 17:48:53] Child process unexpectedly failed with an exit code 9. pid=487361


## Chat Template

As mentioned before, if you do not specify a vision model's `--chat-template`, the server uses Hugging Face's default template, which only supports text.

We list popular vision models with their chat templates:

- [meta-llama/Llama-3.2-Vision](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct) uses `llama_3_vision`.
- [Qwen/Qwen2.5-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct) uses `qwen2-vl`.
- [google/gemma-3-4b-it](https://huggingface.co/google/gemma-3-4b-it) uses `gemma-it`.
- [openbmb/MiniCPM-V](https://huggingface.co/openbmb/MiniCPM-V) uses `minicpmv`.
- [deepseek-ai/deepseek-vl2](https://huggingface.co/deepseek-ai/deepseek-vl2) uses `deepseek-vl2`.
- [LlaVA-OneVision](https://huggingface.co/lmms-lab/llava-onevision-qwen2-7b-ov) uses `chatml-llava`.
- [LLaVA-NeXT](https://huggingface.co/collections/lmms-lab/llava-next-6623288e2d61edba3ddbf5ff) uses `chatml-llava`.
- [Llama3-LLaVA-NeXT](https://huggingface.co/lmms-lab/llama3-llava-next-8b) uses `llava_llama_3`.
- [LLaVA-v1.5 / 1.6](https://huggingface.co/liuhaotian/llava-v1.6-34b) uses `vicuna_v1.1`.